In [ ]:
!pip install transformers==4.0.0
!pip install tensorflow
!pip install keras
!pip install datasets
!pip install Sentencepiece
!pip install -q torch==1.4.0 -f https://download.pytorch.org/whl/cu101/torch_stable.html
    
!pip install wandb

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [2]:
import transformers
import torch
import keras
from datasets import load_dataset, concatenate_datasets

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from functions import *

from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer

/opt/conda/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Classify synthetic data with confidence Intervals

In [3]:
gpt_train = pd.read_csv("2-synthetic-only/synthetic_train_gpt_labels_as_qs.csv", index_col="Unnamed: 0")
#gpt_test = pd.read_csv("synthetic_test_gpt_labels_as_qs.csv", index_col="Unnamed: 0")
sentences_train = gpt_train["Sentence"].to_list()
#sentences_test = gpt_test["Sentence"].to_list()

In [4]:
splitted_train = np.array_split(sentences_train, 10)

In [5]:
splitted_train[0]

array([' Ich denke, ich habe mich schon immer ein bisschen zu viel gefühlt.',
       ' Ich finde es wichtig mein Gewicht zu reduzieren, da ich einige gesundheitliche Probleme habe.',
       ' Ich muss wieder zurück zu meinen Wurzeln gehen.', ...,
       ' Ich weiß einfach nicht, was ich tun soll.', ' Hallo!',
       ' Ich vermeide Fett, Zucker, Salz und Alkohol.'], dtype='<U256')

In [6]:
from transformers import TextClassificationPipeline

cuda = torch.device('cuda')
model = AutoModelForSequenceClassification.from_pretrained("classifs/forum_only/model", local_files_only=True).cuda()
tokenizer = AutoTokenizer.from_pretrained("bert-base-german-cased")
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)

In [7]:
label_0_scores = []
label_1_scores = []
label_2_scores = []

for entry in splitted_train:
    with pipe.device_placement():
        torch.cuda.empty_cache()
        predictions_train = pipe(list(entry), truncation=True, batch_size=4)
        for entry in predictions_train:
            label_0_scores.append(entry[0]['score'])
            label_1_scores.append(entry[1]['score'])
            label_2_scores.append(entry[2]['score'])

In [8]:
#len(label_0_scores)
pred_train_df = pd.DataFrame({"Sentence": sentences_train, "0_R":label_0_scores, "1_TS": label_1_scores, "2_C": label_2_scores })

In [9]:
pred_train_df.to_csv("gpt_predictions/8_percent_synthetic_predicted_confidence.csv")

In [4]:
splitted_test = np.array_split(sentences_test, 2)

In [7]:
label_0_scores = []
label_1_scores = []
label_2_scores = []

for entry in splitted_test:
    print(entry)
    with pipe.device_placement():
        torch.cuda.empty_cache()
        predictions_train = pipe(list(entry), truncation=True, batch_size=4)
        for entry in predictions_train:
            label_0_scores.append(entry[0]['score'])
            label_1_scores.append(entry[1]['score'])
            label_2_scores.append(entry[2]['score'])
    
pred_test_df = pd.DataFrame({"Sentence": sentences_test, "0_R":label_0_scores, "1_TS": label_1_scores, "2_C": label_2_scores })

[' Ich wünsche mir mehr Kontrolle über meine Ernährung.'
 ' Ich habe mich ein wenig gesünder ernährt.'
 ' Welche Möglichkeiten siehst du, um etwas gegen dieses Gefühl der Anstrengung zu unternehmen?'
 ...
 ' Ich würde gerne wieder mehr Sport treiben, aber ich weiß nicht, wie ich das anfangen soll.'
 ' Ich müsste mehr Zeit haben um mich zu bewegen.'
 ' Ich habe einige Wochen zuvor begonnen, meine Ernährung umzustellen.']
[' Ich bin so müde von der ganzen Arbeit, die ich in meinen Körper investiere und ich möchte endlich Ergebnisse sehen.'
 ' Ich möchte gesund leben und mich auch so fühlen.'
 ' Auch die Geschwindigkeit, mit der ich esse, könnte ich einschränken.'
 ... ' Hallo!' ' Meine Ernährung hat sich sehr verbessert.'
 ' Ich könnte mein Essen einfach nur ansehen und dann wüsste ich, wie viel ich esse.']


In [10]:
pred_test_df.to_csv("synthetic_test_predicted_confidence.csv")

NameError: name 'pred_test_df' is not defined